# Making a corner plot of free parameters

This notebook shows an example of making a corner plot of free model parameters from two COOLEST files/models. A chain file must be contained in the directory of each COOLEST file. In this case we plot the mass model and external shear parameters (they must match between the two COOLEST files, otherwise an exception is raised).

_Note: this notebook is aiming to reproduce fig. 12 from Vernardos & Koopmans (2022)._

__authors__: @gvernard

__created on__: 20/07/23

__last update__: 20/07/23

In [1]:
%config InlineBackend.figure_format = 'retina'

import os
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm, TwoSlopeNorm
from pprint import pprint

from coolest.api.analysis import Analysis
from coolest.api.plotting import ModelPlotter, MultiModelPlotter, plot_corner
from coolest.api import util

ModuleNotFoundError: No module named 'getdist'

In [ ]:
def print_info(coolest_object):
    source_index = 2  # index of the source galaxy in the list of `lensing entities`
    print("Lensing entities:", [type(le).__name__ for le in coolest_object.lensing_entities])
    print("Source light model:", [type(m).__name__ for m in coolest_object.lensing_entities[source_index].light_model])

## Load a few COOLEST files

We are loading two COOLEST files, the ones corresponding to fig. 12 in Vernardos & Koopmans (2022). We add a quick check to ensure that each file has a chain file associated to it in the 'meta' field.

In [ ]:
coolest_1 = util.get_coolest_object('database/corner/model_1/coolest_vkl', verbose=True, check_external_files=False)
print_info(coolest_1)

coolest_2 = util.get_coolest_object('database/corner/model_2/coolest_vkl', verbose=True, check_external_files=False)
print_info(coolest_2)

## Setup custom coordinates

so that we can evaluate light profiles consistently on that specific coordinates grid. 

In [ ]:
coord_orig = util.get_coordinates(coolest)
x_orig, y_orig = coord_orig.pixel_coordinates
print(coord_orig.plt_extent)

#coord_src = coord_orig.create_new_coordinates(pixel_scale_factor=0.1, grid_shape=(1.42, 1.42))
#x_src, y_src = coord_src.pixel_coordinates
#print(coord_src.plt_extent)

## Compute and plot the two-point correlation function

In [ ]:
analysis = Analysis(coolest, input_path, supersampling=5)

# Analytic profile
x0,y0,err0 = analysis.two_point_correlation(rmax=0.8,normalize=True,entity_selection=[0])

# Spiral galaxy
x1,y1,err1 = analysis.two_point_correlation(rmax=0.8,normalize=True,entity_selection=[1])

# Marger
x2,y2,err2 = analysis.two_point_correlation(rmax=0.8,normalize=True,entity_selection=[2])


# This is to normalize the curves to their maximum in order to obtain exactly the same figure as fig. 3 in Vernardos et al. (2020)
#y0 /= np.amax(y0)
#y1 /= np.amax(y1)
#y2 /= np.amax(y2)


fig, ax = plt.subplots(1, 1, figsize=(8, 5.5))

ax.scatter(x0,y0,marker='o',label="Analytic")
ax.scatter(x1,y1,marker='^',label="Spiral")
ax.scatter(x2,y2,marker='s',label="Merger")

ax.set_xlabel(r"$r$ [arcsec]")
ax.set_ylabel(r"Power")

ax.legend()
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
fig.tight_layout()
plt.show()